# Notebook to calculate Inter Annotator Agreement

### Import the libraries

In [1]:
import sys
import pandas as pd
import collections 
import os
import numpy as np
from itertools import chain
from itertools import combinations
sys.path.append('/home/jkuettel/NLP_spark/src')
sys.path.append('/home/jkuettel/NLP_spark')
from src.experiment_utils.helper_classes import token, span, repository
from src.d02_corpus_statistics.corpus import Corpus
from src.d03_inter_annotator_agreement.inter_annotator_agremment import Inter_Annotator_Agreement, _get_score_article
from definitions import df_annotation_marker
from src.d03_inter_annotator_agreement.inter_annotator_agremment import row_to_span_list, keep_valid_anotations


from definitions import ROOT_DIR


## Small Tutorial

Load the dataframe stat_df

In [2]:
    
dataframe_dir = os.path.join(ROOT_DIR,'data/02_processed_to_dataframe', 'preprocessed_dataframe.pkl')
stat_df = pd.read_pickle(dataframe_dir)
stat_df.head()

Policy  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...          
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...          
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...          
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...          
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...          

                                                                                                 Text  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  article 3\r\ndefinitions\r\nfor the purposes o...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  article 25\r\npenalties\r\nmember states shall...   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...  article 7\r\njoint projects between member sta...   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  article 47\r\nindependence of the transmission...   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  article 14\r\nadjustment of m0 and tm0 values\...   

                                                                                               Tokens  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  [start:0 stop:7 text:article tag_count:0, star...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  [start:0 stop:7 text:article tag_count:0, star...   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...  [start:0 stop:7 text:article tag_count:0, star...   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  [start:0 stop:7 text:article tag_count:0, star...   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  [start:0 stop:7 text:article tag_count:0, star...   

                                                        Article_State  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  CURATION_FINISHED   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  CURATION_FINISHED   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...  CURATION_FINISHED   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  CURATION_FINISHED   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  CURATION_FINISHED   

                                                   Finished_Annotators  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...     [Fride, Onerva]   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...     [Fride, Onerva]   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...    [Alisha, Fabian]   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...    [Alisha, Onerva]   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...    [Onerva, Fabian]   

                                                                                             Curation  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  [annotator:curation layer:Instrumenttypes type...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  [annotator:curation layer:Instrumenttypes type...   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...  [annotator:curation layer:Policydesigncharacte...   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  [annotator:curation layer:Instrumenttypes type...   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  [annotator:curation layer:Instrumenttypes type...   

                                                                                               Onerva  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  [annotator:Onerva layer:Instrumenttypes type:I...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  [annotator:Onerva layer:Instrumenttypes type:I...   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...                                                 []   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  [annotator:Onerva layer:Instrumenttypes type:I...   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  [annotator:Onerva layer:Instrumenttypes type:I...   

                                                                                                Fride  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  [annotator:Fride layer:Instrumenttypes type:In...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  [annotator:Fri

First create a object of class Inter annotator agreement. The constructor takes a stat_df as input, has a optional argument DEBUG where only the first 10 articles are taken to test different functions

In [3]:
test_evaluator = Inter_Annotator_Agreement(stat_df)
test_evaluator_debug = Inter_Annotator_Agreement(stat_df, DEBUG = True)

Inter_Annotator_Agreement is a child class of the Corpus class, so all methods of the Corpus class are available

In [4]:
test_dir = repository(policy = 'EU_32008R1099')
test_evaluator.get_span_list(conditional_rep = test_dir, columns = 'annotators', item = 'tag', value =  'Tech_LowCarbon')

[annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:125 stop:141 text:renewable energy,
 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:393 stop:409 text:renewable energy,
 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:499 stop:515 text:renewable energy,
 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:1276 stop:1292 text:renewable energy,
 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:701 stop:716 text:energy products,
 annotator:Fride layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon start:866 stop:888 text:electricity generation,
 annotator:Onerva layer:Technologyandapplicationspecificity type:TechnologySpecificity tag:Tech_LowCarbon 

To calculate the inter annonator agreement, there are two options


## Append the score to dataframe

This method appends the inter-annotator agreement for each article which at least two valid annoations based on a set of inter-annotator agreement measures. The scores are calculated in parallel, this is the recommended method for computationally intensive scores

In [21]:
scoring_metrics = ['f1_exact', 'f1_tokenwise', 'f1_partial', 'f1_heuristic'] #, 'pygamma']

In [ ]:
test_evaluator.append_total_score_per_article(scoring_metrics)

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


Process ForkPoolWorker-21:
Process ForkPoolWorker-11:
Process ForkPoolWorker-25:
Process ForkPoolWorker-4:
Process ForkPoolWorker-35:
Process ForkPoolWorker-32:
Process ForkPoolWorker-18:
Process ForkPoolWorker-56:
Process ForkPoolWorker-26:
Process ForkPoolWorker-7:
Process ForkPoolWorker-14:
Process ForkPoolWorker-42:
Process ForkPoolWorker-54:
Process ForkPoolWorker-44:
Process ForkPoolWorker-10:
Process ForkPoolWorker-39:
Process ForkPoolWorker-15:
Process ForkPoolWorker-6:
Process ForkPoolWorker-2:
Process ForkPoolWorker-5:
Process ForkPoolWorker-16:
Process ForkPoolWorker-29:
Process ForkPoolWorker-33:
Process ForkPoolWorker-13:
Process ForkPoolWorker-8:
Process ForkPoolWorker-43:
Process ForkPoolWorker-40:
Process ForkPoolWorker-17:
Process ForkPoolWorker-58:
Process ForkPoolWorker-50:
Process ForkPoolWorker-28:
Process ForkPoolWorker-3:
Process ForkPoolWorker-51:
Process ForkPoolWorker-46:
Process ForkPoolWorker-20:
Process ForkPoolWorker-27:
Process ForkPoolWorker-57:
Process 

There is also a normal implementation which does not use parallel

In [6]:
test_evaluator.append_total_score_per_article_no_parallel(scoring_metrics)

100%|██████████| 448/448 [00:00<00:00, 1355.26it/s]


Checking out the dataframe now:

In [7]:
test_evaluator.df.head()

Policy  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...          
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...          
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...          
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...          
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...          

                                                                                                 Text  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  article 3\r\ndefinitions\r\nfor the purposes o...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  article 25\r\npenalties\r\nmember states shall...   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...  article 7\r\njoint projects between member sta...   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  article 47\r\nindependence of the transmission...   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  article 14\r\nadjustment of m0 and tm0 values\...   

                                                                                               Tokens  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  [start:0 stop:7 text:article tag_count:0, star...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  [start:0 stop:7 text:article tag_count:0, star...   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...  [start:0 stop:7 text:article tag_count:0, star...   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  [start:0 stop:7 text:article tag_count:0, star...   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  [start:0 stop:7 text:article tag_count:0, star...   

                                                        Article_State  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  CURATION_FINISHED   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  CURATION_FINISHED   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...  CURATION_FINISHED   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  CURATION_FINISHED   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  CURATION_FINISHED   

                                                   Finished_Annotators  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...     [Fride, Onerva]   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...     [Fride, Onerva]   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...    [Alisha, Fabian]   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...    [Alisha, Onerva]   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...    [Onerva, Fabian]   

                                                                                             Curation  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  [annotator:curation layer:Instrumenttypes type...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  [annotator:curation layer:Instrumenttypes type...   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...  [annotator:curation layer:Policydesigncharacte...   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  [annotator:curation layer:Instrumenttypes type...   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  [annotator:curation layer:Instrumenttypes type...   

                                                                                               Onerva  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  [annotator:Onerva layer:Instrumenttypes type:I...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  [annotator:Onerva layer:Instrumenttypes type:I...   
EU_32009L0028_Title_0_Chapter_0_Section_0_Artic...                                                 []   
EU_32019L0944_Title_0_Chapter_6_Section_3_Artic...  [annotator:Onerva layer:Instrumenttypes type:I...   
EU_32019R0631_Title_0_Chapter_0_Section_0_Artic...  [annotator:Onerva layer:Instrumenttypes type:I...   

                                                                                                Fride  \
EU_32006L0032_Title_0_Chapter_1_Section_0_Artic...  [annotator:Fride layer:Instrumenttypes type:In...   
EU_32006L0066_Title_0_Chapter_0_Section_0_Artic...  [annotator:Fri

### Get total score

To retrieve the total score of the corpus, use get_total_score_df() on a dataframes where the scores for individual articles have been calculated.

In [8]:
test_evaluator.get_total_score_df()

{'f1_exact_score': 0.24962194821446082,
 'f1_tokenwise_score': 0.23386478587348017,
 'f1_partial_score': 0.3145298761379569,
 'f1_heuristic_score': 0.3231179223979569}

if only specific scores are required:

In [9]:
test_evaluator.get_total_score_df('f1_exact_score')

{'f1_exact_score': 0.24962194821446082}

or

In [10]:
test_evaluator.get_total_score_df(['f1_exact_score', 'f1_tokenwise_score'])

{'f1_exact_score': 0.24962194821446082,
 'f1_tokenwise_score': 0.23386478587348017}

### Get total score per annotator

In [13]:
test_evaluator.get_score_annotator('Fride')

{'f1_exact_score': 0.4420954808109599,
 'f1_tokenwise_score': 0.41871223291621384,
 'f1_partial_score': 0.5540782020100747,
 'f1_heuristic_score': 0.5717870000010115}

In [14]:
test_evaluator.get_score_annotator('Fride', ['f1_exact_score', 'f1_tokenwise_score'] )

{'f1_exact_score': 0.4420954808109599,
 'f1_tokenwise_score': 0.41871223291621384}

## Get total score based on a spanlist

The inter annotator agreement score can be also calculated from a spanlist. For all the spans present, it calculates the inter agreement scores for alle the articls with at least two valid annoations. Can be used to caluclate simmilarity to curation.

In [16]:
test_dir = repository.from_repository_name('EU_32006L0032_Title_0_Chapter_1_Section_0_Article_03')
span_list = test_evaluator_debug.get_span_list(test_dir, ['Onerva', 'Fride'])

In [17]:
test_evaluator_debug.get_score_spanlist(span_list, 'f1_heuristic')

0.5664166174246597

In [18]:
span_list_Onerva_cur = test_evaluator_debug.get_span_list(test_dir, ['Curation', 'Fride'])
span_list_Fride_cur = test_evaluator_debug.get_span_list(test_dir, ['Curation', 'Onerva'])

In [19]:
test_evaluator_debug.get_score_spanlist(span_list_Onerva_cur, 'f1_heuristic')

0.7911308736930657

In [20]:
test_evaluator_debug.get_score_spanlist(span_list_Fride_cur, 'f1_heuristic')

0.6940756370719039

In [6]:
row = test_evaluator.df.loc['EU_32013R0525_Title_0_Chapter_1_Section_0_Article_03']
annotations = row[df_annotation_marker:]
annotations
valid_annotations = keep_valid_anotations(annotations)

In [9]:
rep =  repository.from_repository_name('EU_32013R0525_Title_0_Chapter_1_Section_0_Article_03')
tag_list_2 = test_evaluator.get_span_list(rep, columns='annotators')


In [13]:
len(tag_list_2)

140

In [12]:
len(tag_list_1)

140